# Searching Efficient 3D Architectures with Sparse Point-Voxel Convolution

In this tutorial, we will introduce how to efficiently segment LiDAR point clouds with our pre-trained SPVNAS.

Let's first clone the codebase:

In [ ]:
!git clone https://github.com/mit-han-lab/spvnas.git

import os
import sys
sys.path.append(os.path.join(os.getcwd(), 'spvnas'))

Let's then install some libraries. This step might take around 5 minutes on Google Colab.

In [5]:
!sudo apt-get install libsparsehash-dev 1>/dev/null
!pip install --upgrade plotly 1>/dev/null
!pip install --upgrade torchpack 1>/dev/null
!pip install --upgrade git+https://github.com/mit-han-lab/torchsparse.git 1>/dev/null

  Running command git clone -q https://github.com/mit-han-lab/torchsparse.git /tmp/pip-req-build-lxoy0j7s
  ERROR: Command errored out with exit status 1:
   command: /opt/conda/bin/python3.8 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-req-build-lxoy0j7s/setup.py'"'"'; __file__='"'"'/tmp/pip-req-build-lxoy0j7s/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-1smo531r
       cwd: /tmp/pip-req-build-lxoy0j7s/
  Complete output (156 lines):
  /opt/conda/lib/python3.8/site-packages/setuptools/dist.py:458: UserWarning: Normalizing '2.0.0b' to '2.0.0b0'
    warnings.warn(tmpl.format(**locals()))
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.8
  cr

    ERROR: Command errored out with exit status 1:
     command: /opt/conda/bin/python3.8 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-req-build-lxoy0j7s/setup.py'"'"'; __file__='"'"'/tmp/pip-req-build-lxoy0j7s/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-9e8rdg0_/install-record.txt --single-version-externally-managed --compile --install-headers /opt/conda/include/python3.8/torchsparse
         cwd: /tmp/pip-req-build-lxoy0j7s/
    Complete output (158 lines):
    /opt/conda/lib/python3.8/site-packages/setuptools/dist.py:458: UserWarning: Normalizing '2.0.0b' to '2.0.0b0'
      warnings.warn(tmpl.format(**locals()))
    running install
    running build
    running build_py
    creating build
    c

Let's import some libraries and define constants for visualization:

In [ ]:
import numpy as np

import torch
from torchsparse import SparseTensor
from torchsparse.utils.quantize import sparse_quantize
from torchsparse.utils.collate import sparse_collate

COLOR_MAP = np.array(['#f59664', '#f5e664', '#963c1e', '#b41e50', '#ff0000',
                      '#1e1eff', '#c828ff', '#5a1e96', '#ff00ff', '#ff96ff',
                      '#4b004b', '#4b00af', '#00c8ff', '#3278ff', '#00af00',
                      '#003c87', '#50f096', '#96f0ff', '#0000ff', '#ffffff'])

LABEL_MAP = np.array([19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 0, 1, 19,
                      19, 19, 2, 19, 19, 3, 19, 4, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 5, 6, 7, 19, 19, 19, 19, 19, 19,
                      19, 8, 19, 19, 19, 9, 19, 19, 19, 10, 11, 12, 13,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 14, 15, 16, 19, 19, 19, 19, 19,
                      19, 19, 17, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
                      19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19])

Let's load some real lidar data and pre-process it for inference.

In [ ]:
# Load the LiDAR data and its label
lidar = np.fromfile('spvnas/assets/000000.bin', dtype=np.float32)
label = np.fromfile('spvnas/assets/000000.label', dtype=np.int32)
lidar = lidar.reshape(-1, 4)
label = LABEL_MAP[label & 0xFFFF]

# Filter out ignored points
lidar = lidar[label != 19]
label = label[label != 19]

# Quantize coordinates
coords = np.round(lidar[:, :3] / 0.05)
coords -= coords.min(0, keepdims=1)
feats = lidar

# Filter out duplicate points
coords, indices, inverse = sparse_quantize(coords, return_index=True, return_inverse=True)
coords = torch.tensor(coords, dtype=torch.int)
feats = torch.tensor(feats[indices], dtype=torch.float)

inputs = SparseTensor(coords=coords, feats=feats)
inputs = sparse_collate([inputs]).cuda()

Now, we import the pretrained SPVNAS from our model zoo to run the inference.

In [ ]:
from model_zoo import spvnas_specialized

# Load the pre-trained model from model zoo
model = spvnas_specialized('SemanticKITTI_val_SPVNAS@65GMACs').cuda()
model.eval()

# Run the inference
outputs = model(inputs)
outputs = outputs.argmax(1).cpu().numpy()

# Map the prediction back to original points
outputs = outputs[inverse]

Finally, we visualize the predictions from SPVNAS in an interactive window. Enjoy!

In [ ]:
%matplotlib inline

def configure_plotly_browser_state():
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

import plotly
import plotly.graph_objs as go

trace = go.Scatter3d(
    x=lidar[:, 0],
    y=lidar[:, 1],
    z=lidar[:, 2],
    mode='markers',
    marker={
        'size': 1,
        'opacity': 0.8,
        'color': COLOR_MAP[outputs].tolist(),
    }
)

configure_plotly_browser_state()
plotly.offline.init_notebook_mode(connected=False)

layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
    scene=dict(aspectmode='manual', aspectratio=dict(x=1, y=1, z=0.2))
)

plotly.offline.iplot(go.Figure(data=[trace], layout=layout))